In [2]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.calibration import CalibrationDisplay
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay,DetCurveDisplay,PrecisionRecallDisplay,RocCurveDisplay,classification_report,r2_score
from sklearn.model_selection import GridSearchCV,KFold,cross_validate,train_test_split
from df_after_transform import df_after_transform
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
set_config(display="diagram")

In [18]:
bod_df = pd.read_csv('./input_data_clean/romoBOD_DF.csv')
ceo_df = pd.read_csv('./input_data_clean/romoBOD_DF.csv')

In [19]:
bod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8773 entries, 0 to 8772
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           8773 non-null   int64  
 1   total_director_comp  8773 non-null   float64
 2   year                 8773 non-null   int64  
 3   mkvalt               6922 non-null   float64
 4   size_category        6922 non-null   object 
 5   signature_index      8773 non-null   object 
 6   net_income           3149 non-null   float64
 7   num_employees        8699 non-null   float64
 8   debt_to_equity       8757 non-null   float64
 9   assets_in_place      7321 non-null   float64
 10  capex_by_assets      3147 non-null   float64
 11  return_on_equity     3149 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 822.6+ KB


In [16]:
bod_df.drop('Unnamed: 0', axis = 1)

,Unnamed: 0.1,total_director_comp,year,mkvalt,size_category,signature_index,net_income,num_employees,debt_to_equity,assets_in_place,capex_by_assets,return_on_equity
0,0,902.555,7.606885,1.720190,Small,589b2d914f25e0a1ff8f707f7aa52096,NaN,1.472451,-7.359442,1.064931,0.078205,-0.018774
1,1,902.555,7.607381,2.174025,Small,436a3154f7acadd68f77107364944857,NaN,1.477796,-4.353677,1.042184,0.067511,-0.082984
2,2,902.555,7.607878,2.354188,Medium,dc94df973b47ec03bf615d54bd90e720,NaN,1.470980,-3.943533,1.030753,0.080306,-0.079796
3,3,902.555,7.608374,2.320731,Medium,c2357bce3ae6160bca4e3155a24b1a0a,NaN,1.548437,-16.480776,0.718861,0.073655,-0.043380
4,4,902.555,7.608871,2.309821,Medium,7abb44613cbb465cef913dbfd121155a,2.075213,1.553933,20.658090,0.807452,0.121336,0.065843
...,...,...,...,...,...,...,...,...,...,...,...,...
8768,8768,200.136,7.608374,2.169362,Small,9a2552c4e1a18391ae83d6a74537a7b6,1.233722,0.732099,-23.451613,0.287489,0.010203,0.015657
8769,8769,200.136,7.608871,2.165119,Small,c5aa2527abdb80465777f4eeb1bce3fc,1.642085,0.760837,-416.125000,0.291086,0.025547,0.086909
8770,8770,200.136,7.609367,2.189533,Small,b33c9610b47875a2cebb9d47b97dd758,1.616672,0.806793,88.109375,0.269768,0.015403,0.067343
8771,8771,200.136,7.609862,2.189242,Small,46be5701765385ac27f8a4d0adabf5f6,1.692705,0.806793,18.808473,0.284951,0.018911,0.101940


In [21]:
bod_df.to_csv('./input_data_clean/romoBOD_DF.csv')

In [20]:
bod_df['mkvalt'] = np.log(bod_df.mkvalt)
bod_df['net_income'] = np.log(bod_df.net_income)
bod_df['num_employees'] = np.log(bod_df.num_employees)

/Users/annaharv46/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
bod_df = pd.read_csv('./Input_data/DirectorComp.csv')
bod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49247 entries, 0 to 49246
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GVKEY          49247 non-null  int64  
 1   DIRNBR         49247 non-null  int64  
 2   DIRNAME        49247 non-null  object 
 3   CASH_FEES      49247 non-null  float64
 4   STOCK_AWARDS   49247 non-null  float64
 5   OPTION_AWARDS  49247 non-null  float64
 6   NONEQ_INCENT   49247 non-null  float64
 7   PENSION_CHG    49247 non-null  float64
 8   OTHCOMP        49247 non-null  float64
 9   TOTAL_SEC      49247 non-null  float64
 10  YEAR           49247 non-null  int64  
 11  CONAME         49247 non-null  object 
 12  CUSIP          49247 non-null  object 
 13  EXCHANGE       49247 non-null  object 
 14  ADDRESS        49247 non-null  object 
 15  CITY           49247 non-null  object 
 16  STATE          47165 non-null  object 
 17  ZIP            49134 non-null  object 
 18  SICDES